In [1]:
import socket
import sys
import pandas as pd
import json

We'll cheat a little here - let's read in the data as a CSV, then output the response as a JSON.
In real life, you might get this from an API, or from a Kafka instance. But today we're just trying to demonstrate how to stream through Spark.

In [2]:
def get_flow_data():
    df = pd.read_csv('/home/ds/notebooks/netflow_d02tinyh.csv')
    response = df.to_json(orient='records')
    return response

In [3]:
def send_flows_to_spark(http_resp, tcp_connection):
    obj = json.loads(http_resp)
    for line in obj:
        try:
            src_device = line['SrcDevice']
            print("Source Device: " + src_device)
            print ("------------------------------------------")
            tcp_connection.send(src_device + '\n')
        except Exception as e:
            print("Error: %s" % e)

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting flows.")
resp = get_flow_data()
send_flows_to_spark(resp, conn)

Waiting for TCP connection...
